In [48]:
import pandas as pd
import numpy as np
#import util
import keras as K
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
# import matplotlib.pyplot as plt
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [63]:
#trainData = pd.read_csv('data/train.csv')
trainData = pd.read_csv('data/046ec29ddf80d62e.csv')
#testData = pd.read_csv('data/test.csv')

In [70]:
# here is for feature engineering data process
data = np.array(trainData)
print(data.shape)

(8784, 15)


In [54]:
#trainData['timestamp'] = pd.to_datetime(trainData['timestamp'], unit='s')

In [5]:
data = trainData[trainData['KPI ID']=='046ec29ddf80d62e']
data = data.set_index('timestamp')
data = data.drop(columns=['KPI ID'])
print(data[data['label']==1].count()/data.count())

value    0.009107
label    0.009107
dtype: float64


In [65]:
data = preprocessing.minmax_scale(data)

In [66]:
def splitInputOutput(data,sequence_length, feature_column_start=0, feature_column_end=-1):
    trainInput = []
    trainOutput = []
    for index in range(len(data) - sequence_length):
        trainInput.append(data[index: index + sequence_length,feature_column_start:feature_column_end])
        trainOutput.append(data[index+sequence_length-1,-1])
    return trainInput, trainOutput

In [67]:
trainInput, trainOutput = splitInputOutput(data,sequence_length=30,feature_column_start=0,feature_column_end=-1)

In [68]:
np.array(trainInput).shape

(8754, 30, 14)

In [69]:
np.array(trainOutput).shape

(8754,)

In [71]:
'''
暴力重复过采样至正负1:1，不可取，只做示意
'''
idxRecord = []
for idx in range(len(trainOutput)):
    if trainOutput[idx] == 1:
        idxRecord.append(idx)
overSampleTimes = int(int(len(trainInput)/len(idxRecord) -2)/2)
print('oversample times: ', overSampleTimes)
for i in range(overSampleTimes):
    for idx in idxRecord:
        trainInput.append(trainInput[idx])
        trainOutput.append(trainOutput[idx])
PositiveCount = 0
for idx in range(len(trainOutput)):
    if trainOutput[idx] == 1:
        PositiveCount += 1
print('Positive sample ratio:',PositiveCount/len(trainInput))

oversample times:  53
Positive sample ratio: 0.33246113590888104


In [72]:
def buildModel(input_shape, drop=0.5, lr=1e-2, loss='mse', metrics=[]):
    model = Sequential()
    model.add(Bidirectional(LSTM(64,return_sequences=True), input_shape=input_shape))
    model.add(Dropout(drop))
    model.add(Bidirectional(LSTM(128,return_sequences=True)))
    model.add(Dropout(drop))
    model.add(Bidirectional(LSTM(64,return_sequences=False)))
    model.add(Dense(units=256,activation='relu'))
    model.add(Dropout(drop))
    model.add(Dense(units=1,activation='sigmoid'))
    c = optimizers.adam(lr = lr)
    model.compile(loss="binary_crossentropy", optimizer=c, metrics=metrics)
    return model

In [73]:
trainX = np.array(trainInput)
trainY = np.array(trainOutput)
trainX = K.utils.normalize(trainX)

In [74]:
x_train, x_valid, y_train, y_valid = train_test_split(trainX, trainY, test_size=0.2, random_state=123)

In [ ]:
model = buildModel(input_shape=(None,14),drop=0.5,\
                        lr=1e-2, loss='binary_crossentropy',metrics=['acc'])
# save the best model here (for future use)
#model_checkpoint = ModelCheckpoint(new_save_model_name,monitor='val_my_iou_metric_2', 
#                                       mode = 'max', save_best_only=True, verbose=1)
# reduce the learning rate to get further progress
reduce_lr = ReduceLROnPlateau(monitor='acc', mode = 'max',\
                              factor=0.5, patience=5, min_lr=0.0001, verbose=1)
# early stopping to avoid overfitting
#early_stopping = EarlyStopping(monitor='acc', mode = 'max', patience=20, verbose=1)
epochs = 20
batch_size = 128
history = model.fit(x_train, y_train,
                    validation_data=[x_valid, y_valid], 
                    epochs=epochs,
                    batch_size=batch_size,
                    #callbacks=[ model_checkpoint], 
                    verbose=1)
#plot_history(history,'my_iou_metric')

Train on 10395 samples, validate on 2599 samples
Epoch 1/20
10395/10395 [==============================] - 46s 4ms/step - loss: 0.3241 - acc: 0.8920 - val_loss: 0.1879 - val_acc: 0.9481
Epoch 2/20
10395/10395 [==============================] - 37s 4ms/step - loss: 0.1762 - acc: 0.9461 - val_loss: 0.1658 - val_acc: 0.9519
Epoch 3/20
10395/10395 [==============================] - 41s 4ms/step - loss: 0.0672 - acc: 0.9720 - val_loss: 0.0025 - val_acc: 0.9992
Epoch 4/20
10395/10395 [==============================] - 40s 4ms/step - loss: 0.0192 - acc: 0.9942 - val_loss: 8.9368e-04 - val_acc: 1.0000
Epoch 5/20
10395/10395 [==============================] - 40s 4ms/step - loss: 0.0248 - acc: 0.9926 - val_loss: 0.6335 - val_acc: 0.9130
Epoch 6/20
10395/10395 [==============================] - 41s 4ms/step - loss: 0.0810 - acc: 0.9727 - val_loss: 0.0028 - val_acc: 1.0000
Epoch 7/20
10395/10395 [==============================] - 40s 4ms/step - loss: 0.0073 - acc: 0.9984 - val_loss: 2.6143e-04 - 